In [1]:
import torch
torch.__version__

'1.6.0'

# 使用PyTorch计算梯度数值

PyTorch的Autograd模块实现了深度学习算法的反向传播求导数，

张量（Tensor类）上的所有操作，Autograd都能自动提供微分，简化手动计算导数的复杂过程。

0.4以前的版本，Pytorch用Variable类自动计算所有梯度。Variable类主要包含三个属性：
* data：保存Variable包含的Tensor；
* grad：保存data对应的梯度，grad也是个Variable而不是Tensor，和data形状一样；
* grad_fn：指向一个Function对象，这个Function用来反向传播计算输入的梯度。
0.4起Variable正式合并入Tensor类，通过Variable嵌套实现的自动微分功能已经整合进入Tensor类中。
虽然为了代码的兼容性，还可以使用Variable(tensor)嵌套，但是这个操作其实什么都没做。

代码建议直接使用Tensor类进行操作，官方文档已经将Variable设置成过期模块。Variable类的grad和grad_fn属性已经整合进Tensor类

Tensor类本身就支持使用autograd功能，只需要设置.requries_grad=True

## Autograd

创建张量时，设置requires_grad为Ture，告诉Pytorch对该张量进行自动求导，

PyTorch记录该张量的每一步操作历史并自动计算

In [2]:
x = torch.rand(5, 5, requires_grad=True)
x

tensor([[0.2810, 0.1552, 0.9683, 0.4325, 0.4694],
        [0.4452, 0.6167, 0.4275, 0.3529, 0.6961],
        [0.4412, 0.3087, 0.4232, 0.5188, 0.3209],
        [0.1191, 0.7253, 0.3196, 0.5039, 0.0804],
        [0.5030, 0.9203, 0.0197, 0.9228, 0.3934]], requires_grad=True)

In [3]:
y = torch.rand(5, 5, requires_grad=True)
y

tensor([[0.9283, 0.5328, 0.2041, 0.7443, 0.1791],
        [0.6844, 0.2254, 0.1960, 0.7563, 0.7521],
        [0.8751, 0.4032, 0.0141, 0.6587, 0.3319],
        [0.0858, 0.5516, 0.0913, 0.9577, 0.6419],
        [0.0025, 0.2869, 0.3185, 0.9246, 0.1066]], requires_grad=True)

PyTorch自动追踪和记录对与张量的所有操作，计算完成后调用.backward()自动计算梯度并将计算结果保存到grad属性。

In [4]:
z = torch.sum(x + y)
z

tensor(22.8185, grad_fn=<SumBackward0>)

进行张量操作后，grad_fn已经被赋予一个新的函数，引用一个创建这个Tensor类的Function对象。

Tensor和Function互相连接生成一个**非循环图**，记录并且编码完整的计算历史。

每个张量都有一个.grad_fn属性，如果用户手动创建这个张量，那么这个张量的grad_fn是None。

调用反向传播函数，计算其梯度

## 简单的自动求导

In [5]:
z.backward()
print(x.grad, y.grad)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]) tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])


如果Tensor类表示一个标量（包含一个元素的张量），不需要为backward()指定任何参数，

但是如果它有更多元素，则需要指定一个gradient参数，是形状匹配的张量。

 `z.backward()`相当于`z.backward(torch.tensor(1.))`简写。
 
这种参数常出现在**图像分类的单标签分类**，输出一个标量代表图像的标签。

## 复杂的自动求导

In [6]:
x = torch.rand(5, 5, requires_grad=True)
y = torch.rand(5, 5, requires_grad=True)
z = x**2 + y**3
z

tensor([[0.7077, 0.6606, 1.1397, 0.3069, 0.2019],
        [0.8434, 0.9232, 0.1591, 0.3068, 0.6101],
        [0.1596, 1.0615, 0.4189, 0.8729, 0.4370],
        [0.6869, 1.0105, 1.2285, 0.9861, 0.2764],
        [0.0838, 1.2262, 0.6027, 0.0093, 0.9261]], grad_fn=<AddBackward0>)

In [7]:
torch.ones_like(x)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [8]:
# 返回值不是一个标量，要输入一个大小相同的张量作为参数，用ones_like根据x生成一个张量
z.backward(torch.ones_like(x))
print(x.grad)

tensor([[1.2364, 0.5709, 0.9951, 1.0823, 0.1712],
        [1.6148, 1.7020, 0.7598, 0.1322, 1.4575],
        [0.7866, 1.3320, 0.6802, 1.8294, 0.5635],
        [0.5076, 1.7863, 1.3530, 1.3254, 0.9682],
        [0.3197, 1.9531, 1.5498, 0.1785, 0.7019]])


可以使用with torch.no_grad()上下文管理器，临时禁止对已设置requires_grad=True的张量自动求导。

这个方法在测试集计算准确率时会经常用到，例如：

In [9]:
with torch.no_grad():
    print((x + y * 2).requires_grad)

False


.no_grad()嵌套后，代码不会跟踪历史记录，保存的这部分记录会减少内存使用量，并且会加快少许运算速度。

## Autograd 过程解析

为了说明Pytorch的自动求导原理，尝试分析一下PyTorch源代码。

虽然Pytorch的 Tensor和 TensorBase用CPP实现，但是可用一些Python方法查看这些对象在Python的属性和状态。

Python的 `dir()` 返回参数的属性、方法列表。`z`是一个Tensor变量，看看里面有哪些成员变量。

In [10]:
dir(z)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rfloordiv__',
 '__rmul__',
 '__rpow__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__

返回很多，直接排除Python特殊方法（以__开头和结束的）和私有方法（以_开头的）

直接看几个**主要属性**：
`.is_leaf`：记录是否是叶子节点。
通过这个属性确定变量类型在官方文档所说的“graph leaves”，“leaf variables”，指像`x`，`y`这样手动创建的、而非运算得到的变量，这些变量称为创建变量。

像`z`这样的，是通过计算后得到的结果，称为结果变量。

一个变量是创建变量还是结果变量，通过`.is_leaf`来获取。

In [11]:
print("x.is_leaf=" + str(x.is_leaf))
print("z.is_leaf=" + str(z.is_leaf))

x.is_leaf=True
z.is_leaf=False


`x`是手动创建的没有通过计算，所以他被认为是一个叶子节点也就是一个创建变量，而`z`是通过`x`与`y`的一系列计算得到的，所以不是叶子结点也就是结果变量。

为什么执行`z.backward()`方法，会更新`x.grad`和`y.grad`呢？
`.grad_fn`属性记录这部分操作，虽然`.backward()`方法也是CPP实现，但可以通过Python进行简单探索。

`grad_fn`：记录并且编码完整的计算历史

In [12]:
z.grad_fn

`grad_fn`是一个`AddBackward0`类型变量。

`AddBackward0`这个类也是用Cpp写的，是加法(ADD)的反向传播（Backward）。

In [13]:
dir(z.grad_fn)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_register_hook_dict',
 'metadata',
 'name',
 'next_functions',
 'register_hook',
 'requires_grad']

**`next_functions`**是`grad_fn`的精华

In [14]:
z.grad_fn.next_functions

((<PowBackward0 at 0x127ad1970>, 0), (<PowBackward0 at 0x127ad1940>, 0))

`next_functions`是tuple of tuple of PowBackward0 and int。

为什么是2个tuple？因为操作是`z= x**2+y**3`。`AddBackward0`是相加，前面的操作是乘方 `PowBackward0`。
tuple第一个元素是x相关的操作记录。

In [15]:
xg = z.grad_fn.next_functions[0][0]
xg

In [16]:
dir(xg)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_register_hook_dict',
 'metadata',
 'name',
 'next_functions',
 'register_hook',
 'requires_grad']

继续深挖

In [17]:
x_leaf = xg.next_functions[0][0]
type(x_leaf)

AccumulateGrad

在PyTorch反向图计算中，`AccumulateGrad`类型代表叶子节点类型，也就是计算图终止节点。

`AccumulateGrad`类中有一个`.variable`属性指向叶子节点。

In [18]:
x_leaf.variable

tensor([[0.6182, 0.2854, 0.4976, 0.5412, 0.0856],
        [0.8074, 0.8510, 0.3799, 0.0661, 0.7288],
        [0.3933, 0.6660, 0.3401, 0.9147, 0.2818],
        [0.2538, 0.8931, 0.6765, 0.6627, 0.4841],
        [0.1599, 0.9766, 0.7749, 0.0892, 0.3510]], requires_grad=True)

这个`.variable`的属性就是生成的变量`x`

In [19]:
x

tensor([[0.6182, 0.2854, 0.4976, 0.5412, 0.0856],
        [0.8074, 0.8510, 0.3799, 0.0661, 0.7288],
        [0.3933, 0.6660, 0.3401, 0.9147, 0.2818],
        [0.2538, 0.8931, 0.6765, 0.6627, 0.4841],
        [0.1599, 0.9766, 0.7749, 0.0892, 0.3510]], requires_grad=True)

In [20]:
print("x_leaf.variable 的 id: " + str(id(x_leaf.variable)))
print("x 的 id: " + str(id(x)))

x_leaf.variable 的 id: 4960817600
x 的 id: 4960817600


In [21]:
assert (id(x_leaf.variable) == id(x))

整个规程清晰：

1. 执行 **z.backward()** 时，操作调用z的grad_fn属性，执行求导操作。
2. 求导操作遍历grad_fn的next_functions，取出里面的Function（AccumulateGrad）执行求导操作。这部分是**递归**，直到最后类型为叶子节点。
3. 计算出结果后，将结果保存到对应variable变量引用对象（x和y）的**grad属性**里。
4. 求导结束，所有叶节点的grad变量得到相应**更新**

最终执行完c.backward()后，a和b的grad值得到更新。

## 扩展Autograd

如果要自定义autograd扩展新功能，就要扩展Function类。因为Function用autograd计算结果和梯度，并对操作历史编码。

Function类中最主要方法是`forward()`和`backward()`，分别代表前向传播和反向传播。


一个自定义的Function需要三个方法：

1. __init__ (optional)：如果操作需要额外参数，则要定义这个Function的构造函数，不需要可以忽略。
1. **forward()** ：执行前向传播的计算代码
1. **backward()** ：反向传播时计算梯度的代码。参数个数和forward返回值个数一样，每个参数代表传回到此操作的梯度。        

In [22]:
# 引入Function便于扩展
from torch.autograd.function import Function

In [23]:
# 定义一个乘以常数的操作(输入参数是张量)
# 方法必须是静态方法，要加上@staticmethod
class MulConstant(Function):
    @staticmethod
    def forward(ctx, tensor, constant):
        # ctx 用来保存信息，类似self，并且ctx属性可以在backward中调用
        ctx.constant = constant
        return tensor * constant

    @staticmethod
    def backward(ctx, grad_output):
        # 返回的参数要与输入的参数一样.
        # 第一个输入为3x3的张量，第二个为一个常数
        # 常数的梯度必须是 None.
        return grad_output, None

定义完新操作后，进行测试

In [24]:
a = torch.rand(3, 3, requires_grad=True)
# b为a的元素乘以5
b = MulConstant.apply(a, 5)
print("a:" + str(a))
print("b:" + str(b))  

a:tensor([[0.1827, 0.5063, 0.9923],
        [0.7839, 0.7627, 0.4652],
        [0.3048, 0.0986, 0.1137]], requires_grad=True)
b:tensor([[0.9135, 2.5315, 4.9613],
        [3.9193, 3.8137, 2.3259],
        [1.5240, 0.4932, 0.5686]], grad_fn=<MulConstantBackward>)


反向传播，返回值不是标量，所以`backward`方法需要参数

In [25]:
b.backward(torch.ones_like(a))

In [26]:
a.grad

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])